Start by importing numpy

In [3]:
import numpy as np

This is the whole data set

In [4]:
fileInput = np.genfromtxt('./1.txt', delimiter=',')
fileInput.shape

(235762, 43)

We'll slice it into our input, output and train, test, dev sets

In [5]:
X_train = fileInput[:230000, :42]
Y_train = fileInput[:230000, 42:]
X_test = fileInput[230000:235000, :42]
Y_test = fileInput[230000:235000, 42:]
X_dev = fileInput[235000:235762, :42]
Y_dev = fileInput[235000:235762, 42:]

In [6]:
print('input train set:', X_train.shape)
print('output train set:', Y_train.shape)
print('input test set:', X_test.shape)
print('output test set:', Y_test.shape)
print('input dev set:', X_dev.shape)
print('output dev set:', Y_dev.shape)

input train set: (230000, 42)
output train set: (230000, 1)
input test set: (5000, 42)
output test set: (5000, 1)
input dev set: (762, 42)
output dev set: (762, 1)


In [7]:
#pip install keras==2.3.1
print(X_train[0:5,:])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [-1. -1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]]


In [12]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, ZeroPadding2D, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

#### Build model

In [15]:
def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    ### START CODE HERE ###
    
    # Step 1: Dense layer (≈4 lines)
    X = ZeroPadding2D(4)(X_input)
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 1, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = Conv2D(32, (5, 5), strides = (1, 1), name = 'conv1')(X)
    X = BatchNormalization(axis = 1, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 1, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = Dense(32, activation='relu')(X)
    X = Dropout(.8)(X)
    X = Dense(1, activation='sigmoid')(X)

    ### END CODE HERE ###

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [16]:
filModel = model(X_train.shape[1:])

ValueError: Shape must be rank 4 but is rank 2 for '{{node zero_padding2d_2/Pad}} = Pad[T=DT_FLOAT, Tpaddings=DT_INT32](input_3, zero_padding2d_2/Pad/paddings)' with input shapes: [?,42], [4,2].

In [29]:
filModel.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [32]:
filModel.fit(x=X_train, y=Y_train, epochs=10, batch_size=64)

Epoch 1/10
230000/230000 [==============================] - 12s 53us/step - loss: 0.6757 - accuracy: 0.5736
Epoch 2/10
230000/230000 [==============================] - 11s 49us/step - loss: 0.6725 - accuracy: 0.5805
Epoch 3/10
230000/230000 [==============================] - 11s 48us/step - loss: 0.6710 - accuracy: 0.5836
Epoch 4/10
230000/230000 [==============================] - 11s 49us/step - loss: 0.6703 - accuracy: 0.5841
Epoch 5/10
230000/230000 [==============================] - 11s 46us/step - loss: 0.6694 - accuracy: 0.5865
Epoch 6/10
230000/230000 [==============================] - 10s 45us/step - loss: 0.6691 - accuracy: 0.5868
Epoch 7/10
230000/230000 [==============================] - 10s 44us/step - loss: 0.6691 - accuracy: 0.5863
Epoch 8/10
230000/230000 [==============================] - 10s 44us/step - loss: 0.6684 - accuracy: 0.5882
Epoch 9/10
230000/230000 [==============================] - 11s 46us/step - loss: 0.6680 - accuracy: 0.5887
Epoch 10/10
230000/230000 [=

In [33]:
### START CODE HERE ### (1 line)
preds = filModel.evaluate(x = X_test, y = Y_test)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

5000/5000 [==============================] - 0s 22us/step

Loss = 0.6763212266921997
Test Accuracy = 0.5863999724388123


In [48]:
x = np.reshape(X_dev[0,:], (1, 42))
print(x.shape)
print('prediction:', filModel.predict(x))
print('correct:', Y_dev[0])

(1, 42)
prediction: [[0.5207819]]
correct: [1.]
